In [91]:
from Crypto.Cipher import AES
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import numpy as np
import base64

In [92]:
public_space = {}
message_space = {}

def generate_keys_rsa() -> (bytes,bytes):
    random_generator = Random.new().read
    key = RSA.generate(2048, random_generator)
    public_key = key.publickey().exportKey()
    private_key = key.exportKey()
    return (public_key,private_key)

def encrypt_rsa(message:bytes, public_key:bytes) -> bytes:
    cipher = PKCS1_OAEP.new(RSA.importKey(public_key))
    encrypted_message = cipher.encrypt(message)
    return encrypted_message

def decrypt_rsa(message:bytes, private_key:bytes) -> bytes:
    cipher_dec = PKCS1_OAEP.new(RSA.importKey(private_key))
    decrypted_message = cipher_dec.decrypt(message)
    return decrypted_message

def encrypt_aes(text : bytes, key : bytes, iv : bytes) -> bytes:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext_padded = pad(text, AES.block_size)
    return cipher.encrypt(plaintext_padded)

def decrypt_aes(key : bytes, iv : bytes, text : bytes) -> bytes:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext_padded = cipher.decrypt(text)
    plaintext = unpad(plaintext_padded, AES.block_size)
    return plaintext

# def send_message(message:str, public_key) -> bytes:


In [93]:
def handshake(name:str, family_name:str, receiver_id:str):
    full_name = name + "|" + family_name
    public_k = public_space[receiver_id]
    encoded_name = encrypt_rsa(message=full_name.encode(), public_key=public_k)
    message_space[receiver_id].append(encoded_name)

id_A = "A"
public_key_A, private_key_A = generate_keys_rsa()
public_space["A"] = public_key_A
message_space["A"] = []

name_A = "name_A"
family_name_A = "family_A"

id_B = "B"
public_key_B, private_key_B = generate_keys_rsa()
public_space["B"] = public_key_B
message_space["B"] = []

name_B = "name_B"
family_name_B = "family_B"

handshake(name=name_A, family_name=family_name_A, receiver_id="B")
handshake(name=name_B, family_name=family_name_B, receiver_id="A")

### USER A :
B's name and sends encoded "B name:send number" to B

In [94]:
Bs_name = decrypt_rsa(message_space["A"][0], private_key_A).decode().split("|")
decoded_A = []
print(Bs_name)
message_space["B"].append(encrypt_rsa(f"{Bs_name[0]}:send number".encode(), public_key_B))

['name_B', 'family_B']


### USER B : 
decodes user A's message and shows it and then sends encrypted number "a" to user A

In [95]:
As_name = decrypt_rsa(message_space["B"][0], private_key_B).decode().split("|")
decoded_B = [decrypt_rsa(message_space["B"][-1], private_key_B).decode()]
print("Received Message --->",decoded_B[-1])
print(As_name)
a = 5
pub_key_A = RSA.import_key(public_key_A)
pub_key_B = RSA.import_key(public_key_B)
pr_key_B = RSA.import_key(private_key_B)
message_space["A"].append(pow(a ,pub_key_B.e,pub_key_B.n))


Received Message ---> name_B:send number
['name_A', 'family_A']


### USER A :
chooses a number "b" and sends a^b to user B

In [96]:
b = 3
pwr = pow(message_space["A"][-1],b)
message_space["B"].append(pwr)

### USER B :
get user A's power.<br>
Sends it back to A.

In [97]:
A_power = round(np.log(pow(message_space["B"][-1],pr_key_B.d,pr_key_B.n))/np.log(a),5)
# print("A's Selected Power ---> ",A_power)
message_space["A"].append(encrypt_rsa(f"{As_name[0]}: power is {A_power}".encode(), public_key_A))

### USER A : 
Decrypts the message and shows it.<br>
If the power is right, send B an AES key

In [98]:
shown_message_pwr = decrypt_rsa(message_space["A"][-1], private_key_A).decode()
print("Decoded message from B ---> ",shown_message_pwr)
symmteric_key = get_random_bytes(32)
message_space["B"].append(encrypt_rsa(symmteric_key, public_key_B))

Decoded message from B --->  name_A: power is 3.0


### USER B :
encrypts "hello" with the symmetric key that A sent.

In [99]:
symmteric_key = decrypt_rsa(message_space["B"][-1], private_key_B)
iv            = get_random_bytes(16)
message = "hello"
encrypted_message = encrypt_aes(text=message.encode(), key=symmteric_key, iv=iv)
message_space["A"].append([encrypted_message, iv])

### USER A :
opens the message and sees it : 

In [100]:
decrypted_aes_message = decrypt_aes(key = symmteric_key, 
                                    iv = message_space["A"][-1][1], text= message_space["A"][-1][0])
print(decrypted_aes_message.decode())

hello
